In [59]:
import cv2
import numpy as np

# NUMBER PLATE DETECTION

In [76]:
# Load Yolo
net = cv2.dnn.readNet("Number_Plate.weights", "custom-yolov4-detector.cfg")
classes = []
with open("Number_Plate.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

In [82]:
# Loading image
img = cv2.imread("test.jpg")
img = cv2.resize(img, None, fx=0.4, fy=0.4)
height, width, channels = img.shape
# Detecting objects
blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
net.setInput(blob)
outs = net.forward(output_layers)

In [83]:
# Showing informations on the screen
class_ids = []
confidences = []
boxes = []
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            # Object detected
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            # Rectangle coordinates
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

In [84]:
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

In [85]:
font = cv2.FONT_HERSHEY_PLAIN
for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        #color = colors[i]
        cv2.rectangle(img, (x, y), (x + w, y + h), (255,0,0), 2)
        cv2.putText(img, label, (x, y + 30), font, 1, (255,0,0), 1)
cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Using Keras OCR

In [97]:
import matplotlib.pyplot as plt
import keras_ocr
import cv2

In [98]:
pipeline = keras_ocr.pipeline.Pipeline()

Looking for C:\Users\vishw\.keras-ocr\craft_mlt_25k.h5
Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
Looking for C:\Users\vishw\.keras-ocr\crnn_kurapan.h5


In [117]:
def TestOCR(image):
    # Load Yolo
    net = cv2.dnn.readNet("Number_Plate.weights", "custom-yolov4-detector.cfg")
    classes = []
    with open("Number_Plate.names", "r") as f:
        classes = [line.strip() for line in f.readlines()]
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    colors = np.random.uniform(0, 255, size=(len(classes), 3))
    # Loading image
    img = cv2.imread("Test_Images/"+image)
    img = cv2.resize(img, None, fx=0.4, fy=0.4)
    height, width, channels = img.shape
    # Detecting objects
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Showing informations on the screen
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    seg = None

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    #Number list

    n_list = []

    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            seg = img[y:y+h, x:x+w ,:]
            #Getting the segment value
            images = [keras_ocr.tools.read(seg)]
            prediction = pipeline.recognize(images)
            print(prediction)

    cv2.imshow("Image", seg)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [119]:
TestOCR("9.jpg")

[[('hroibj', array([[ 20.,  32.],
       [113.,  32.],
       [113.,  65.],
       [ 20.,  65.]], dtype=float32)), ('temp', array([[120.,  34.],
       [145.,  34.],
       [145.,  48.],
       [120.,  48.]], dtype=float32)), ('6662', array([[142.,  34.],
       [202.,  34.],
       [202.,  64.],
       [142.,  64.]], dtype=float32)), ('superon', array([[ 49.,  65.],
       [130.,  65.],
       [130.,  75.],
       [ 49.,  75.]], dtype=float32)), ('e', array([[24., 66.],
       [42., 66.],
       [42., 74.],
       [24., 74.]], dtype=float32)), ('hyundal', array([[138.,  66.],
       [200.,  66.],
       [200.,  75.],
       [138.,  75.]], dtype=float32))]]


# Vechicle Detection

In [94]:
colors

array([[ 90.57196129, 125.91946378, 194.99096973],
       [229.38927347, 248.53021654,  53.795779  ],
       [ 40.80069422, 233.89561374, 195.7679089 ],
       [244.73607131, 192.8773119 ,  31.31855761],
       [ 27.11958284, 147.79201115,  78.19786236],
       [115.17216819, 199.7142421 ,  87.10555625],
       [163.7003947 , 235.3012834 ,  50.77567483],
       [135.56395967,   7.56734598,  21.79856936],
       [  9.64897764, 225.97674817, 193.9983056 ]])

In [86]:
# Load Yolo
net = cv2.dnn.readNet("Vehicles.weights", "custom-yolov4-detector.cfg")
classes = []
with open("Vehicles.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

In [95]:
# Loading image
img = cv2.imread("test_2.png")
img = cv2.resize(img, None, fx=0.4, fy=0.4)
height, width, channels = img.shape
# Detecting objects
blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
net.setInput(blob)
outs = net.forward(output_layers)

In [88]:
# Showing informations on the screen
class_ids = []
confidences = []
boxes = []
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            # Object detected
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            # Rectangle coordinates
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

In [89]:
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

In [90]:
font = cv2.FONT_HERSHEY_PLAIN
for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        #color = colors[i]
        cv2.rectangle(img, (x, y), (x + w, y + h), (255,0,0), 2)
        cv2.putText(img, label, (x, y + 30), font, 1, (255,0,0), 1)
cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [91]:
boxes

[]

In [92]:
indexes

()